In [1]:
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-geometric

     |████████████████████████████████| 7.9 MB 5.3 MB/s 
     |████████████████████████████████| 3.5 MB 2.9 MB/s 
     |████████████████████████████████| 370 kB 5.5 MB/s 
     |████████████████████████████████| 482 kB 30.1 MB/s 
     |████████████████████████████████| 41 kB 405 kB/s 


In [2]:
!wget -q https://github.com/abojchevski/sparse_smoothing/raw/master/sparse_smoothing/sparsegraph.py
!mkdir data && cd data && wget -q https://github.com/abojchevski/sparse_smoothing/raw/master/data/cora_ml.npz

In [2]:
from statsmodels.stats.proportion import proportion_confint
from sparsegraph import SparseGraph
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
seed = 5
np.random.seed(seed) # Set the random seed of numpy for the data split.
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

In [5]:
# Hyper-parameters

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
p_plus  = 0.001
p_minus = 0.06
alpha = 0.01
r_a = 3
r_d = 4
n_samples = 100
n_samples_eval = 50
batch_size = 10

In [6]:
def load_and_standardize(file_name):
    """
    Run gust.standardize() + make the attributes binary.
    Parameters
    ----------
    file_name
        Name of the file to load.
    Returns
    -------
    graph: gust.SparseGraph
        The standardized graph
    """
    with np.load(file_name, allow_pickle=True) as loader:
        loader = dict(loader)
        if 'type' in loader:
            del loader['type']
        graph = SparseGraph.from_flat_dict(loader)
    
    graph.standardize()
    
    # binarize
    graph._flag_writeable(True)
    graph.adj_matrix[graph.adj_matrix != 0] = 1
    graph.attr_matrix[graph.attr_matrix != 0] = 1
    graph._flag_writeable(False)
    return graph

In [7]:
# Load dataset
graph = load_and_standardize('data/cora_ml.npz')
n, d = graph.attr_matrix.shape
nc = graph.labels.max() + 1
print('number nodes: ', n, 'feature dim: ', d, 'num_classes: ', nc)

number nodes:  2810 feature dim:  2879 num_classes:  7


In [9]:
def perturb_phi(X, p_plus, p_minus, batchsize=1):
    n, d = X.shape
    X = X[None,:,:].repeat(batchsize, axis=0)
    binomial_minus = np.random.binomial(1, p_minus, size=(batchsize,n,d))
    binomial_plus  = np.random.binomial(1, p_plus,  size=(batchsize,n,d))
    epsilon = np.where(X==0, binomial_minus, binomial_plus)
    output = np.logical_xor(X, epsilon).astype('int')
    return output

# outs = perturb_phi(graph.attr_matrix.toarray(), p_plus, p_minus, batchsize=2)
# outs.shape

In [10]:
msk = np.random.rand(n) < 0.8
train_mask = msk
test_mask = ~msk

In [11]:
# Train the given model on the given graph for num_epochs
def train(model, data, train_mask, test_mask, p_plus, p_minus, num_epochs, nsamples=1, batchsize=1, device=None):
    
    # Important: add self-edges so a node depends on itself
    adj = torch.tensor(data.adj_matrix.toarray())
    adj += torch.eye(adj.shape[0])
    adj = adj.to(device)
    data_labels = torch.tensor(data.labels[train_mask]).to(device)

    # Set up the loss and the optimizer
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    # A utility function to compute the accuracy
    def get_acc(outs, y, mask):
        return (outs[mask].argmax(dim=1) == y[mask]).sum().float() / mask.sum()

    best_acc_val = -1
    X = data.attr_matrix.toarray()
    model.to(device)
    model.train()
    for epoch in range(num_epochs):
        # Zero grads -> forward pass -> compute loss -> backprop
        optimizer.zero_grad()
        outs = torch.zeros([n, nc], dtype=torch.float, device=device)
        if p_plus + p_minus > 0:
            for i in range(0, nsamples, batchsize):
                X_batch = perturb_phi(X, p_plus, p_minus, min(batchsize, nsamples-i))
                X_batch = torch.FloatTensor(X_batch).to(device)
                outs += model(X_batch, adj).sum(0)
            outs /= nsamples
        else:
            outs = model(torch.FloatTensor(X).to(device), adj)
        
        loss = loss_fn(outs[train_mask], data_labels)
        loss.backward()
        optimizer.step()

        # Compute accuracies, print only if this is the best result so far
        acc_test = get_acc(outs.cpu(), torch.tensor(data.labels), test_mask)
        acc_train = get_acc(outs.cpu(), torch.tensor(data.labels), train_mask)
        if acc_test > best_acc_val:
            best_acc_val = acc_test
            print(f'[Epoch {epoch+1}/{num_epochs}] Loss: {loss} | train: {acc_train:.3f} | Test: {acc_test:.3f}')

In [12]:
class GNNModel(nn.Module):
    def __init__(self, num_layers=2, sz_in=2879, sz_hid=64, sz_out=7):
        super().__init__()

        
        self.layers = nn.ModuleList([
                                     nn.Linear(
                                         sz_in if i == 0 else sz_hid, sz_out if i == num_layers-1 else sz_hid
                                     ) for i in range(num_layers)
        ])
        

    def forward(self, fts, adj):
        """
        Args:
            fts: [N, D] 
            adj: [N, N] 

        Returns:
            new_fts: [N, 7]
        """
        deg = adj.sum(axis=1, keepdim=True) # Degree of nodes, shape [N, 1]
        
        for i, layer in enumerate(self.layers):
            fts = layer(fts)
            fts = adj @ fts / deg 
            if i < len(self.layers) -1:
                fts = torch.relu(fts)
        
        return fts
model = GNNModel()

In [13]:
train(model, graph, train_mask, test_mask, p_plus, p_minus, num_epochs=100, nsamples=n_samples, batchsize=batch_size, device=device)

[Epoch 1/100] Loss: 1.9336978197097778 | train: 0.267 | Test: 0.280
[Epoch 2/100] Loss: 3.5132558345794678 | train: 0.276 | Test: 0.288
[Epoch 3/100] Loss: 1.872841238975525 | train: 0.383 | Test: 0.422
[Epoch 15/100] Loss: 1.6278094053268433 | train: 0.396 | Test: 0.434
[Epoch 16/100] Loss: 1.5941722393035889 | train: 0.439 | Test: 0.456
[Epoch 17/100] Loss: 1.5565263032913208 | train: 0.507 | Test: 0.519
[Epoch 18/100] Loss: 1.5172582864761353 | train: 0.561 | Test: 0.568
[Epoch 19/100] Loss: 1.479609489440918 | train: 0.627 | Test: 0.617
[Epoch 20/100] Loss: 1.4394080638885498 | train: 0.669 | Test: 0.645
[Epoch 21/100] Loss: 1.3955748081207275 | train: 0.696 | Test: 0.675
[Epoch 22/100] Loss: 1.3488879203796387 | train: 0.700 | Test: 0.684
[Epoch 23/100] Loss: 1.2995319366455078 | train: 0.697 | Test: 0.688
[Epoch 29/100] Loss: 1.0256916284561157 | train: 0.709 | Test: 0.690
[Epoch 30/100] Loss: 0.9846044182777405 | train: 0.727 | Test: 0.700
[Epoch 31/100] Loss: 0.9453845024108887

In [14]:
def predict(model, data, train_mask, test_mask, p_plus, p_minus, nsamples=1, batchsize=1, device=None):
    
    # Important: add self-edges so a node depends on itself
    adj = torch.tensor(data.adj_matrix.toarray())
    adj += torch.eye(adj.shape[0])
    adj = adj.to(device)

    model.eval()
    votes = torch.zeros((n, nc), dtype=torch.long, device=device)
    
    with torch.no_grad():
        X = data.attr_matrix.toarray()
        if p_plus + p_minus > 0:
            for i in tqdm(range(0, nsamples, batchsize), leave=False):
                X_batch = perturb_phi(X, p_plus, p_minus, min(batchsize, nsamples-i))
                X_batch = torch.FloatTensor(X_batch).to(device)
                outs = model(X_batch, adj)
                votes += F.one_hot(outs.argmax(2), nc).sum(0)
        else:
            outs = model(torch.FloatTensor(X).to(device), adj)
            votes += F.one_hot(outs.argmax(1), nc)
    return votes.cpu().numpy()

In [15]:
votes = predict(model.to(device), graph, train_mask, test_mask, p_plus, p_minus, nsamples=n_samples, batchsize=batch_size, device=device)
pre_votes = predict(model.to(device), graph, train_mask, test_mask, p_plus, p_minus, nsamples=n_samples_eval, batchsize=batch_size, device=device)
votes.sum(0)

array([35677, 37422, 43883, 39370, 79339, 15007, 30302])

In [16]:
def p_lower_from_votes(votes, pre_votes, alpha, n_samples):
    """
    Estimate a lower bound on the probability of the majority class using a Binomial confidence interval.
    Parameters
    ----------
    votes: array_like [n_nodes, n_classes]
        Votes per class for each sample
    pre_votes: array_like [n_nodes, n_classes]
        Votes (based on fewer samples) to determine the majority (and the second best) class
    alpha : float
        Significance level
    n_samples : int
        Number of MC samples
    Returns
    -------
    p_lower: array-like [n_nodes]
        Lower bound on the probability of the majority class
    """
    # Multiple by 2 since we are only need a single side
    n_best = votes[np.arange(votes.shape[0]), pre_votes.argmax(1)]
    p_lower = proportion_confint(
        n_best, n_samples, alpha=2 * alpha, method="beta")[0]

    return p_lower

def compute_likelihood_ratio(q, r_a, r_d, p_plus, p_minus):
    return ((p_plus/(1-p_minus)) ** (q-r_d)) * ((p_minus/(1-p_plus)) ** (q-r_a))

def likelihood_ratio_and_R(p_plus, p_minus, r_a, r_d):
    max_q = r_a + r_d
    likelihood_ratio = np.zeros(max_q)
    R = np.zeros(max_q)
    R[0] = 1

    for q in range(max_q):
        likelihood_ratio[q] = compute_likelihood_ratio(q, r_a, r_d, p_plus, p_minus)
        if q == 0:
            continue
        for i in range(1, q):
            T_i = (r_a * ((p_plus/(1-p_plus)) ** i)) + (r_d * ((p_minus/(1-p_minus)) ** i))
            R[q] += (-1**(-i+1)) * T_i * R[q-i]
        R[q] /= q
         
    return R, likelihood_ratio

def binary_certificate(p_plus, p_minus, r_a, r_d, p_lower):
    
    all_rho = np.zeros((n, r_a, r_d))

    for n_ in range(len(p_lower)):
        for ra in range(r_a):
            for rd in range(r_d):
                p = 0
                rho = 0
                if p_plus+p_minus < 1:
                    start = 0
                    end = r_a + r_d
                else:
                    start = r_a + r_d
                    end = 0

                R, likelihood_ratio = likelihood_ratio_and_R(p_plus, p_minus, r_a, r_d)
                PB_phi_x = R * ((1-p_plus) ** ra) * ((1-p_minus) ** rd)
                PB_phi_x_neigh = PB_phi_x/likelihood_ratio
                    
                for q in range(start, end):
                    if p + PB_phi_x[q] > p_lower[n_]:
                        break
                    else:
                        p += PB_phi_x[q]
                        rho += PB_phi_x_neigh[q]
                
                    if p_lower[n_] - p > 0:
                        rho += (p_lower[n_] - p)/likelihood_ratio[q]

                all_rho[n_, ra, rd] = rho
    return all_rho

p_lower = p_lower_from_votes(votes, pre_votes, alpha, n_samples_eval)
all_rho = binary_certificate(p_plus, p_minus, r_a, r_d, p_lower)
# all_rho

In [1]:
heatmap = (all_rho>0.5).mean(0)
sns.set_context('talk')
sns.heatmap(heatmap, 
            cmap='Greys',
            vmin=0, vmax=1, square=True, cbar_kws={"shrink": .5})
plt.xlim(0, heatmap.shape[1])
plt.ylim(0, heatmap.shape[0])
plt.xlabel('$r_d$ radius')
plt.ylabel('$r_a$ radius')
plt.show()